<a href="https://colab.research.google.com/github/My1stK8s/skillfactoryds/blob/master/nbasearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

In [15]:
!pip install optuna
!pip install tabnet
!pip install pytorch_tabnet
!pip install catboost


In [16]:
# !pip install xgboost


In [17]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [18]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import  XGBClassifier
import torch
import optuna
from optuna import Trial, visualization
from pytorch_tabnet.tab_model import TabNetClassifier
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix,roc_auc_score,confusion_matrix,accuracy_score
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score




In [20]:
# Таблица метрик для сравнения результатов каждой модели
metrics = ['accuracy', 'precision', 'recall', 'f1_score']
summary = pd.DataFrame(index = metrics)
def metrics(summary,col,y_test,y_pred):
    values = [accuracy_score(y_test,y_pred), precision_score(y_test,y_pred), recall_score(y_test,y_pred), f1_score(y_test,y_pred),]
    summary[col] = values
    return summary

# Confusion matrix
def conf_mat(y_test, y_pred):
    P = np.sum(y_test==1)
    N = np.sum(y_test==0)
    print('Positive: {0}, Negative: {1}'.format(P,N))
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(tp, fp) 
    print(fn, tn)

# Date preproc
def preproc(X):
    data= X['Date'].str.split('-',expand=True)
    data.columns=['month','day','year','end_year']
    X = pd.concat([X,data],axis = 1)
    X['month'] = X['month'].astype(int)
    X['day'] = X['day'].astype(int)
    X['year'] = X['year'].astype(int)
    X['end_year'] = X['end_year'].astype(int)
    return X

In [21]:
   # pip install pytorch_tabnet
torch.cuda.is_available()

False

In [22]:
df = pd.read_csv('/content/drive/MyDrive/data/allseasons2007-22.csv')
df.drop(['Score',  'Unnamed: 0', 'TEAM_NAME', 'TEAM_NAME.1', 'Date.1', 'OU-Cover', 'OU','TEAM_ID','TEAM_ID.1'],
          axis=1, inplace=True)
df.drop_duplicates(keep=False)

,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,...,FT_PCT.1,OREB.1,DREB.1,REB.1,AST.1,TOV.1,STL.1,BLK.1,BLKA.1,PF.1,PFD.1,PTS.1,PLUS_MINUS.1,GP_RANK.1,W_RANK.1,L_RANK.1,W_PCT_RANK.1,MIN_RANK.1,FGM_RANK.1,FGA_RANK.1,FG_PCT_RANK.1,FG3M_RANK.1,FG3A_RANK.1,FG3_PCT_RANK.1,FTM_RANK.1,FTA_RANK.1,FT_PCT_RANK.1,OREB_RANK.1,DREB_RANK.1,REB_RANK.1,AST_RANK.1,TOV_RANK.1,STL_RANK.1,BLK_RANK.1,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Home-Team-Win
0,1,1,0,1.000,48.0,37.0,76.0,0.487,6.0,16.0,0.375,22.0,43.0,0.512,17.0,24.0,41.0,20.0,19.0,12.0,7.0,8.0,23.0,32.0,102.0,3.0,24,8,1,1,5,15,27,5,20,20,15,10,1,29,...,0.639,16.0,30.0,46.0,18.0,17.0,5.5,6.0,8.0,30.0,22.5,91.0,-11.0,2,20,23,20,5,14,7,20,25,21,24,28,27,27,4,20,10,26,20,23,10,24,30,15,26,24,1
1,2,1,1,0.500,48.0,31.5,75.0,0.420,10.0,20.0,0.500,24.0,34.0,0.706,9.5,28.5,38.0,15.5,11.0,3.5,5.5,6.0,21.0,25.5,97.0,-2.5,2,8,12,12,5,29,28,21,3,13,3,7,5,19,...,0.767,10.5,32.5,43.0,20.5,12.5,8.5,6.0,3.0,18.5,21.0,103.5,17.5,2,2,1,1,5,15,26,6,17,18,14,8,11,11,17,14,15,16,6,12,10,3,2,21,9,3,0
2,2,2,0,1.000,50.5,35.5,87.5,0.406,10.0,22.0,0.455,22.0,28.5,0.772,13.5,39.5,53.0,21.0,15.5,10.5,9.5,7.5,25.5,22.5,103.0,5.5,2,2,1,1,1,21,6,25,3,7,8,10,13,8,...,0.464,11.0,36.5,47.5,18.5,18.0,5.5,6.5,8.5,18.5,16.0,82.5,-6.5,2,20,23,20,5,24,8,26,12,25,4,30,29,30,13,6,9,24,23,23,7,27,2,30,30,19,1
3,2,1,1,0.500,50.5,29.0,75.0,0.387,7.5,22.0,0.341,25.0,29.5,0.847,11.0,30.0,41.0,21.5,18.0,6.5,6.5,4.0,26.5,24.5,90.5,-14.0,2,8,12,12,1,30,28,29,9,7,17,4,12,3,...,0.909,8.0,29.0,37.0,25.5,11.5,9.0,4.5,7.5,19.5,19.5,106.0,23.0,2,2,1,1,5,5,16,3,1,14,1,21,28,1,26,22,26,3,3,9,24,22,7,22,4,1,0
4,1,1,0,1.000,48.0,36.0,79.0,0.456,4.0,15.0,0.267,25.0,31.0,0.806,14.0,38.0,52.0,20.0,15.0,6.0,4.0,10.0,23.0,25.0,101.0,7.0,24,8,1,1,5,20,21,15,26,21,27,4,9,6,...,0.822,6.0,31.0,37.0,18.5,13.0,5.0,9.0,3.0,21.0,22.0,93.0,5.5,2,8,12,12,5,25,30,14,17,10,23,16,19,4,29,18,26,24,7,25,3,3,10,18,24,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18274,42,16,26,0.381,48.2,42.7,93.0,0.459,11.0,31.0,0.353,14.1,19.1,0.741,11.3,34.9,46.2,27.5,12.5,7.5,5.3,5.4,17.5,18.4,110.5,-0.8,10,23,24,24,12,2,1,14,27,28,13,30,28,26,...,0.809,8.6,33.6,42.2,22.8,12.1,8.0,5.4,4.5,19.3,19.1,107.2,1.5,22,9,7,10,3,19,26,12,29,29,16,2,9,5,30,23,30,23,1,8,5,11,13,18,20,11,0
18275,43,24,19,0.558,48.2,38.8,85.5,0.454,11.9,36.1,0.329,16.1,20.8,0.773,9.2,34.4,43.6,24.0,12.7,7.0,3.8,3.6,19.6,20.1,105.5,3.1,5,9,11,11,14,24,27,19,17,11,25,18,19,16,...,0.733,13.3,34.7,48.0,24.9,13.3,10.2,6.4,6.3,19.2,19.2,111.4,3.7,1,3,4,3,24,5,2,18,28,24,24,15,6,29,1,13,1,10,10,1,1,30,12,15,7,6,1
18276,41,22,19,0.537,48.2,40.6,86.9,0.467,12.7,36.5,0.347,14.6,18.8,0.780,8.9,34.2,43.1,27.2,13.2,7.0,4.0,5.0,19.1,19.3,108.5,1.3,22,12,11,12,10,10,17,7,10,9,17,29,30,10,...,0.781,10.2,32.3,42.4,22.4,14.8,7.8,4.7,5.0,21.5,20.4,102.7,-9.5,10,29,29,29,12,29,15,29,22,13,28,7,8,9,16,29,29,27,25,9,21,20,29,3,28,30,1
18277,42,20,22,0.476,48.4,40.5,91.7,0.442,14.3,41.5,0.344,16.0,21.0,0.758,12.5,32.7,45.1,24.5,14.9,8.5,5.6,5.5,22.2,20.3,111.3,0.6,10,17,17,18,4,11,4,24,3,1,20,21,15,22,...,0.832,10.1,35.1,45.2,26.1,14.3,7.1,5.4,5.0,20.2,19.4,113.4,3.5,22,4,3,4,10,3,11,1,26,27,12,4,13,1,18,8,11,6,21,21,7,21,27,13,2,8,1


In [23]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Using {}".format(DEVICE))

Using cpu


In [24]:
dd = preproc(df) 


In [25]:
dd.drop_duplicates(keep=False)


,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,...,AST.1,TOV.1,STL.1,BLK.1,BLKA.1,PF.1,PFD.1,PTS.1,PLUS_MINUS.1,GP_RANK.1,W_RANK.1,L_RANK.1,W_PCT_RANK.1,MIN_RANK.1,FGM_RANK.1,FGA_RANK.1,FG_PCT_RANK.1,FG3M_RANK.1,FG3A_RANK.1,FG3_PCT_RANK.1,FTM_RANK.1,FTA_RANK.1,FT_PCT_RANK.1,OREB_RANK.1,DREB_RANK.1,REB_RANK.1,AST_RANK.1,TOV_RANK.1,STL_RANK.1,BLK_RANK.1,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Home-Team-Win,month,day,year,end_year
0,1,1,0,1.000,48.0,37.0,76.0,0.487,6.0,16.0,0.375,22.0,43.0,0.512,17.0,24.0,41.0,20.0,19.0,12.0,7.0,8.0,23.0,32.0,102.0,3.0,24,8,1,1,5,15,27,5,20,20,15,10,1,29,...,18.0,17.0,5.5,6.0,8.0,30.0,22.5,91.0,-11.0,2,20,23,20,5,14,7,20,25,21,24,28,27,27,4,20,10,26,20,23,10,24,30,15,26,24,1,11,2,2007,8
1,2,1,1,0.500,48.0,31.5,75.0,0.420,10.0,20.0,0.500,24.0,34.0,0.706,9.5,28.5,38.0,15.5,11.0,3.5,5.5,6.0,21.0,25.5,97.0,-2.5,2,8,12,12,5,29,28,21,3,13,3,7,5,19,...,20.5,12.5,8.5,6.0,3.0,18.5,21.0,103.5,17.5,2,2,1,1,5,15,26,6,17,18,14,8,11,11,17,14,15,16,6,12,10,3,2,21,9,3,0,11,2,2007,8
2,2,2,0,1.000,50.5,35.5,87.5,0.406,10.0,22.0,0.455,22.0,28.5,0.772,13.5,39.5,53.0,21.0,15.5,10.5,9.5,7.5,25.5,22.5,103.0,5.5,2,2,1,1,1,21,6,25,3,7,8,10,13,8,...,18.5,18.0,5.5,6.5,8.5,18.5,16.0,82.5,-6.5,2,20,23,20,5,24,8,26,12,25,4,30,29,30,13,6,9,24,23,23,7,27,2,30,30,19,1,11,2,2007,8
3,2,1,1,0.500,50.5,29.0,75.0,0.387,7.5,22.0,0.341,25.0,29.5,0.847,11.0,30.0,41.0,21.5,18.0,6.5,6.5,4.0,26.5,24.5,90.5,-14.0,2,8,12,12,1,30,28,29,9,7,17,4,12,3,...,25.5,11.5,9.0,4.5,7.5,19.5,19.5,106.0,23.0,2,2,1,1,5,5,16,3,1,14,1,21,28,1,26,22,26,3,3,9,24,22,7,22,4,1,0,11,2,2007,8
4,1,1,0,1.000,48.0,36.0,79.0,0.456,4.0,15.0,0.267,25.0,31.0,0.806,14.0,38.0,52.0,20.0,15.0,6.0,4.0,10.0,23.0,25.0,101.0,7.0,24,8,1,1,5,20,21,15,26,21,27,4,9,6,...,18.5,13.0,5.0,9.0,3.0,21.0,22.0,93.0,5.5,2,8,12,12,5,25,30,14,17,10,23,16,19,4,29,18,26,24,7,25,3,3,10,18,24,11,1,11,2,2007,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18274,42,16,26,0.381,48.2,42.7,93.0,0.459,11.0,31.0,0.353,14.1,19.1,0.741,11.3,34.9,46.2,27.5,12.5,7.5,5.3,5.4,17.5,18.4,110.5,-0.8,10,23,24,24,12,2,1,14,27,28,13,30,28,26,...,22.8,12.1,8.0,5.4,4.5,19.3,19.1,107.2,1.5,22,9,7,10,3,19,26,12,29,29,16,2,9,5,30,23,30,23,1,8,5,11,13,18,20,11,0,1,23,2021,22
18275,43,24,19,0.558,48.2,38.8,85.5,0.454,11.9,36.1,0.329,16.1,20.8,0.773,9.2,34.4,43.6,24.0,12.7,7.0,3.8,3.6,19.6,20.1,105.5,3.1,5,9,11,11,14,24,27,19,17,11,25,18,19,16,...,24.9,13.3,10.2,6.4,6.3,19.2,19.2,111.4,3.7,1,3,4,3,24,5,2,18,28,24,24,15,6,29,1,13,1,10,10,1,1,30,12,15,7,6,1,1,23,2021,22
18276,41,22,19,0.537,48.2,40.6,86.9,0.467,12.7,36.5,0.347,14.6,18.8,0.780,8.9,34.2,43.1,27.2,13.2,7.0,4.0,5.0,19.1,19.3,108.5,1.3,22,12,11,12,10,10,17,7,10,9,17,29,30,10,...,22.4,14.8,7.8,4.7,5.0,21.5,20.4,102.7,-9.5,10,29,29,29,12,29,15,29,22,13,28,7,8,9,16,29,29,27,25,9,21,20,29,3,28,30,1,1,23,2021,22
18277,42,20,22,0.476,48.4,40.5,91.7,0.442,14.3,41.5,0.344,16.0,21.0,0.758,12.5,32.7,45.1,24.5,14.9,8.5,5.6,5.5,22.2,20.3,111.3,0.6,10,17,17,18,4,11,4,24,3,1,20,21,15,22,...,26.1,14.3,7.1,5.4,5.0,20.2,19.4,113.4,3.5,22,4,3,4,10,3,11,1,26,27,12,4,13,1,18,8,11,6,21,21,7,21,27,13,2,8,1,1,23,2021,22


In [26]:
dd

,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,...,AST.1,TOV.1,STL.1,BLK.1,BLKA.1,PF.1,PFD.1,PTS.1,PLUS_MINUS.1,GP_RANK.1,W_RANK.1,L_RANK.1,W_PCT_RANK.1,MIN_RANK.1,FGM_RANK.1,FGA_RANK.1,FG_PCT_RANK.1,FG3M_RANK.1,FG3A_RANK.1,FG3_PCT_RANK.1,FTM_RANK.1,FTA_RANK.1,FT_PCT_RANK.1,OREB_RANK.1,DREB_RANK.1,REB_RANK.1,AST_RANK.1,TOV_RANK.1,STL_RANK.1,BLK_RANK.1,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Home-Team-Win,month,day,year,end_year
0,1,1,0,1.000,48.0,37.0,76.0,0.487,6.0,16.0,0.375,22.0,43.0,0.512,17.0,24.0,41.0,20.0,19.0,12.0,7.0,8.0,23.0,32.0,102.0,3.0,24,8,1,1,5,15,27,5,20,20,15,10,1,29,...,18.0,17.0,5.5,6.0,8.0,30.0,22.5,91.0,-11.0,2,20,23,20,5,14,7,20,25,21,24,28,27,27,4,20,10,26,20,23,10,24,30,15,26,24,1,11,2,2007,8
1,2,1,1,0.500,48.0,31.5,75.0,0.420,10.0,20.0,0.500,24.0,34.0,0.706,9.5,28.5,38.0,15.5,11.0,3.5,5.5,6.0,21.0,25.5,97.0,-2.5,2,8,12,12,5,29,28,21,3,13,3,7,5,19,...,20.5,12.5,8.5,6.0,3.0,18.5,21.0,103.5,17.5,2,2,1,1,5,15,26,6,17,18,14,8,11,11,17,14,15,16,6,12,10,3,2,21,9,3,0,11,2,2007,8
2,2,2,0,1.000,50.5,35.5,87.5,0.406,10.0,22.0,0.455,22.0,28.5,0.772,13.5,39.5,53.0,21.0,15.5,10.5,9.5,7.5,25.5,22.5,103.0,5.5,2,2,1,1,1,21,6,25,3,7,8,10,13,8,...,18.5,18.0,5.5,6.5,8.5,18.5,16.0,82.5,-6.5,2,20,23,20,5,24,8,26,12,25,4,30,29,30,13,6,9,24,23,23,7,27,2,30,30,19,1,11,2,2007,8
3,2,1,1,0.500,50.5,29.0,75.0,0.387,7.5,22.0,0.341,25.0,29.5,0.847,11.0,30.0,41.0,21.5,18.0,6.5,6.5,4.0,26.5,24.5,90.5,-14.0,2,8,12,12,1,30,28,29,9,7,17,4,12,3,...,25.5,11.5,9.0,4.5,7.5,19.5,19.5,106.0,23.0,2,2,1,1,5,5,16,3,1,14,1,21,28,1,26,22,26,3,3,9,24,22,7,22,4,1,0,11,2,2007,8
4,1,1,0,1.000,48.0,36.0,79.0,0.456,4.0,15.0,0.267,25.0,31.0,0.806,14.0,38.0,52.0,20.0,15.0,6.0,4.0,10.0,23.0,25.0,101.0,7.0,24,8,1,1,5,20,21,15,26,21,27,4,9,6,...,18.5,13.0,5.0,9.0,3.0,21.0,22.0,93.0,5.5,2,8,12,12,5,25,30,14,17,10,23,16,19,4,29,18,26,24,7,25,3,3,10,18,24,11,1,11,2,2007,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18274,42,16,26,0.381,48.2,42.7,93.0,0.459,11.0,31.0,0.353,14.1,19.1,0.741,11.3,34.9,46.2,27.5,12.5,7.5,5.3,5.4,17.5,18.4,110.5,-0.8,10,23,24,24,12,2,1,14,27,28,13,30,28,26,...,22.8,12.1,8.0,5.4,4.5,19.3,19.1,107.2,1.5,22,9,7,10,3,19,26,12,29,29,16,2,9,5,30,23,30,23,1,8,5,11,13,18,20,11,0,1,23,2021,22
18275,43,24,19,0.558,48.2,38.8,85.5,0.454,11.9,36.1,0.329,16.1,20.8,0.773,9.2,34.4,43.6,24.0,12.7,7.0,3.8,3.6,19.6,20.1,105.5,3.1,5,9,11,11,14,24,27,19,17,11,25,18,19,16,...,24.9,13.3,10.2,6.4,6.3,19.2,19.2,111.4,3.7,1,3,4,3,24,5,2,18,28,24,24,15,6,29,1,13,1,10,10,1,1,30,12,15,7,6,1,1,23,2021,22
18276,41,22,19,0.537,48.2,40.6,86.9,0.467,12.7,36.5,0.347,14.6,18.8,0.780,8.9,34.2,43.1,27.2,13.2,7.0,4.0,5.0,19.1,19.3,108.5,1.3,22,12,11,12,10,10,17,7,10,9,17,29,30,10,...,22.4,14.8,7.8,4.7,5.0,21.5,20.4,102.7,-9.5,10,29,29,29,12,29,15,29,22,13,28,7,8,9,16,29,29,27,25,9,21,20,29,3,28,30,1,1,23,2021,22
18277,42,20,22,0.476,48.4,40.5,91.7,0.442,14.3,41.5,0.344,16.0,21.0,0.758,12.5,32.7,45.1,24.5,14.9,8.5,5.6,5.5,22.2,20.3,111.3,0.6,10,17,17,18,4,11,4,24,3,1,20,21,15,22,...,26.1,14.3,7.1,5.4,5.0,20.2,19.4,113.4,3.5,22,4,3,4,10,3,11,1,26,27,12,4,13,1,18,8,11,6,21,21,7,21,27,13,2,8,1,1,23,2021,22


In [27]:
dd.drop(columns=["Date"],inplace=True)

In [28]:
dd

,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,...,AST.1,TOV.1,STL.1,BLK.1,BLKA.1,PF.1,PFD.1,PTS.1,PLUS_MINUS.1,GP_RANK.1,W_RANK.1,L_RANK.1,W_PCT_RANK.1,MIN_RANK.1,FGM_RANK.1,FGA_RANK.1,FG_PCT_RANK.1,FG3M_RANK.1,FG3A_RANK.1,FG3_PCT_RANK.1,FTM_RANK.1,FTA_RANK.1,FT_PCT_RANK.1,OREB_RANK.1,DREB_RANK.1,REB_RANK.1,AST_RANK.1,TOV_RANK.1,STL_RANK.1,BLK_RANK.1,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Home-Team-Win,month,day,year,end_year
0,1,1,0,1.000,48.0,37.0,76.0,0.487,6.0,16.0,0.375,22.0,43.0,0.512,17.0,24.0,41.0,20.0,19.0,12.0,7.0,8.0,23.0,32.0,102.0,3.0,24,8,1,1,5,15,27,5,20,20,15,10,1,29,...,18.0,17.0,5.5,6.0,8.0,30.0,22.5,91.0,-11.0,2,20,23,20,5,14,7,20,25,21,24,28,27,27,4,20,10,26,20,23,10,24,30,15,26,24,1,11,2,2007,8
1,2,1,1,0.500,48.0,31.5,75.0,0.420,10.0,20.0,0.500,24.0,34.0,0.706,9.5,28.5,38.0,15.5,11.0,3.5,5.5,6.0,21.0,25.5,97.0,-2.5,2,8,12,12,5,29,28,21,3,13,3,7,5,19,...,20.5,12.5,8.5,6.0,3.0,18.5,21.0,103.5,17.5,2,2,1,1,5,15,26,6,17,18,14,8,11,11,17,14,15,16,6,12,10,3,2,21,9,3,0,11,2,2007,8
2,2,2,0,1.000,50.5,35.5,87.5,0.406,10.0,22.0,0.455,22.0,28.5,0.772,13.5,39.5,53.0,21.0,15.5,10.5,9.5,7.5,25.5,22.5,103.0,5.5,2,2,1,1,1,21,6,25,3,7,8,10,13,8,...,18.5,18.0,5.5,6.5,8.5,18.5,16.0,82.5,-6.5,2,20,23,20,5,24,8,26,12,25,4,30,29,30,13,6,9,24,23,23,7,27,2,30,30,19,1,11,2,2007,8
3,2,1,1,0.500,50.5,29.0,75.0,0.387,7.5,22.0,0.341,25.0,29.5,0.847,11.0,30.0,41.0,21.5,18.0,6.5,6.5,4.0,26.5,24.5,90.5,-14.0,2,8,12,12,1,30,28,29,9,7,17,4,12,3,...,25.5,11.5,9.0,4.5,7.5,19.5,19.5,106.0,23.0,2,2,1,1,5,5,16,3,1,14,1,21,28,1,26,22,26,3,3,9,24,22,7,22,4,1,0,11,2,2007,8
4,1,1,0,1.000,48.0,36.0,79.0,0.456,4.0,15.0,0.267,25.0,31.0,0.806,14.0,38.0,52.0,20.0,15.0,6.0,4.0,10.0,23.0,25.0,101.0,7.0,24,8,1,1,5,20,21,15,26,21,27,4,9,6,...,18.5,13.0,5.0,9.0,3.0,21.0,22.0,93.0,5.5,2,8,12,12,5,25,30,14,17,10,23,16,19,4,29,18,26,24,7,25,3,3,10,18,24,11,1,11,2,2007,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18274,42,16,26,0.381,48.2,42.7,93.0,0.459,11.0,31.0,0.353,14.1,19.1,0.741,11.3,34.9,46.2,27.5,12.5,7.5,5.3,5.4,17.5,18.4,110.5,-0.8,10,23,24,24,12,2,1,14,27,28,13,30,28,26,...,22.8,12.1,8.0,5.4,4.5,19.3,19.1,107.2,1.5,22,9,7,10,3,19,26,12,29,29,16,2,9,5,30,23,30,23,1,8,5,11,13,18,20,11,0,1,23,2021,22
18275,43,24,19,0.558,48.2,38.8,85.5,0.454,11.9,36.1,0.329,16.1,20.8,0.773,9.2,34.4,43.6,24.0,12.7,7.0,3.8,3.6,19.6,20.1,105.5,3.1,5,9,11,11,14,24,27,19,17,11,25,18,19,16,...,24.9,13.3,10.2,6.4,6.3,19.2,19.2,111.4,3.7,1,3,4,3,24,5,2,18,28,24,24,15,6,29,1,13,1,10,10,1,1,30,12,15,7,6,1,1,23,2021,22
18276,41,22,19,0.537,48.2,40.6,86.9,0.467,12.7,36.5,0.347,14.6,18.8,0.780,8.9,34.2,43.1,27.2,13.2,7.0,4.0,5.0,19.1,19.3,108.5,1.3,22,12,11,12,10,10,17,7,10,9,17,29,30,10,...,22.4,14.8,7.8,4.7,5.0,21.5,20.4,102.7,-9.5,10,29,29,29,12,29,15,29,22,13,28,7,8,9,16,29,29,27,25,9,21,20,29,3,28,30,1,1,23,2021,22
18277,42,20,22,0.476,48.4,40.5,91.7,0.442,14.3,41.5,0.344,16.0,21.0,0.758,12.5,32.7,45.1,24.5,14.9,8.5,5.6,5.5,22.2,20.3,111.3,0.6,10,17,17,18,4,11,4,24,3,1,20,21,15,22,...,26.1,14.3,7.1,5.4,5.0,20.2,19.4,113.4,3.5,22,4,3,4,10,3,11,1,26,27,12,4,13,1,18,8,11,6,21,21,7,21,27,13,2,8,1,1,23,2021,22


In [29]:
train3000 = dd[15000:18000]
X3000 = train3000.drop(['Home-Team-Win'], axis=1).values
y3000 = train3000['Home-Team-Win'].values

In [30]:
# test  all 279 next match
test279 = dd[18000:18279]
X279 = test279.drop(['Home-Team-Win'],axis=1).values
y279 =  test279['Home-Team-Win'].values


In [31]:
train18000=dd[:18000]
X18000 = train18000.drop(['Home-Team-Win'], axis=1).values
y18000 = train18000['Home-Team-Win'].values

In [32]:
X18000.shape,y18000.shape,X3000.shape,y3000.shape,X279.shape,y279.shape

((18000, 108), (18000,), (3000, 108), (3000,), (279, 108), (279,))

In [33]:
# наивная модель последние  2 сезона 

In [34]:
clfn3000 = TabNetClassifier(verbose=0)
clfn3000.fit(X_train=X3000, y_train=y3000)  

No early stopping will be performed, last training weights will be used.


In [35]:
# тест 18000 последних  игр
y_pred3000 = clfn3000.predict(X3000)
metrics(summary,'clfn3000',y3000,y_pred3000),conf_mat(y3000, y_pred3000)

Positive: 1659, Negative: 1341
966 359
693 982


(           clfn3000
 accuracy   0.649333
 precision  0.729057
 recall     0.582278
 f1_score   0.647453, None)

In [36]:
# тест 18000 последних  игр
y_pred279 = clfn3000.predict(X279)
metrics(summary,'clfn3000-279',y279,y_pred279),conf_mat(y279, y_pred279)

Positive: 153, Negative: 126
84 38
69 88


(           clfn3000  clfn3000-279
 accuracy   0.649333      0.616487
 precision  0.729057      0.688525
 recall     0.582278      0.549020
 f1_score   0.647453      0.610909, None)

In [37]:
# наивная модель все сезоны 

In [38]:
clfn18000 = TabNetClassifier(verbose=0)
clfn18000.fit(X_train=X18000, y_train=y18000)  

No early stopping will be performed, last training weights will be used.


In [39]:
# тест 18000 последних  игр
y_pred18000 = clfn18000.predict(X18000)
metrics(summary,'clfn18000',y18000,y_pred18000),conf_mat(y18000, y_pred18000)

Positive: 10594, Negative: 7406
8706 3031
1888 4375


(           clfn3000  clfn3000-279  clfn18000
 accuracy   0.649333      0.616487   0.726722
 precision  0.729057      0.688525   0.741757
 recall     0.582278      0.549020   0.821786
 f1_score   0.647453      0.610909   0.779723, None)

In [40]:
# тест 18000 последних  игр
y_pred279 = clfn18000.predict(X279)
metrics(summary,'clfn18000-279',y279,y_pred279),conf_mat(y279, y_pred279)

Positive: 153, Negative: 126
107 57
46 69


(           clfn3000  clfn3000-279  clfn18000  clfn18000-279
 accuracy   0.649333      0.616487   0.726722       0.630824
 precision  0.729057      0.688525   0.741757       0.652439
 recall     0.582278      0.549020   0.821786       0.699346
 f1_score   0.647453      0.610909   0.779723       0.675079, None)

In [41]:
def Objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_d = trial.suggest_int("n_d", 4, 64, step=1)
    n_a = trial.suggest_int("n_a", 4, 64, step=1)
    n_steps = trial.suggest_int("n_steps", 2, 10, step=1)
    gamma = trial.suggest_float("gamma", 0.8, 2, step=0.1)
    n_shared = trial.suggest_int("n_shared", 1, 10, step=1)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    
    tabnet_params = dict(n_d=n_d, n_a=n_a, n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type=mask_type, n_shared=n_shared,
                     scheduler_params=dict(mode="max",
                                           # changing sheduler patience to be lower than early stopping patience 
                                           min_lr=2e-2,
                                           factor=0.5,
                                           ),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     device_name = DEVICE,
                     verbose=0,
                     ) #early stopping
    kfs = KFold(n_splits=10,random_state=42,shuffle=True)
    CV_score_array=[]
    for train_index, test_index in kfs.split(X18000,y18000):
        X_train, X_valid = X18000[train_index], X18000[test_index]
        y_train, y_valid = y18000[train_index], y18000[test_index]
        clf = TabNetClassifier(**tabnet_params)
        clf.fit(X_train=X_train, y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  patience= 50,
                  max_epochs=trial.suggest_int('epochs', 250,300, step = 50),
                  batch_size=2048,
                  virtual_batch_size = 47 ,  
                  eval_metric=['accuracy'], 
                  num_workers=0,
                  weights=1,
                  drop_last=False)
        CV_score_array.append(clf.best_cost)
    avg = np.mean(CV_score_array)
    return avg

In [42]:
study = optuna.create_study(direction="maximize", study_name='TabNet optimization')
study.optimize(Objective, timeout=6*60)


[I 2022-02-03 21:09:34,763] A new study created in memory with name: TabNet optimization



Early stopping occurred at epoch 99 with best_epoch = 49 and best_val_0_accuracy = 0.71833
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 130 with best_epoch = 80 and best_val_0_accuracy = 0.70444
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 142 with best_epoch = 92 and best_val_0_accuracy = 0.70278
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 141 with best_epoch = 91 and best_val_0_accuracy = 0.71111
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 71 with best_epoch = 21 and best_val_0_accuracy = 0.7
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 91 with best_epoch = 41 and best_val_0_accuracy = 0.715
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 217 with best_epoch = 167 and best_val_0_accuracy = 0.70333
Best weights from best epoch are automa

[I 2022-02-03 23:43:45,792] Trial 0 finished with value: 0.7056111111111111 and parameters: {'mask_type': 'entmax', 'n_d': 28, 'n_a': 64, 'n_steps': 4, 'gamma': 1.3, 'n_shared': 9, 'lambda_sparse': 1.6016541518387349e-06, 'epochs': 300}. Best is trial 0 with value: 0.7056111111111111.


In [43]:
TabNet_params = study.best_params
print(TabNet_params)

{'mask_type': 'entmax', 'n_d': 28, 'n_a': 64, 'n_steps': 4, 'gamma': 1.3, 'n_shared': 9, 'lambda_sparse': 1.6016541518387349e-06, 'epochs': 300}


In [43]:
{'mask_type': 'entmax', 'n_d': 28, 'n_a': 64, 'n_steps': 4, 'gamma': 1.3, 'n_shared': 9, 'lambda_sparse': 1.6016541518387349e-06, 'epochs': 300}


In [46]:
final_params = dict(
    mask_type=TabNet_params['mask_type'],
    n_d=TabNet_params['n_d'],
    n_a=TabNet_params['n_a'],
    n_steps=TabNet_params['n_steps'],
    gamma=TabNet_params['gamma'],
    lambda_sparse=TabNet_params['lambda_sparse'],
    n_shared=TabNet_params['n_shared'],
    optimizer_fn=torch.optim.Adam)
clf = TabNetClassifier(**final_params,verbose=0)
clf.fit(X_train=X18000, y_train=y18000,
        max_epochs=TabNet_params['epochs'],
        patience= 50,
        batch_size=2048,
        virtual_batch_size=47)

No early stopping will be performed, last training weights will be used.


In [47]:
# тест 18000 последних  игр
y_pred18000 = clf.predict(X18000)
metrics(summary,'clf18000',y18000,y_pred18000),conf_mat(y18000, y_pred18000)

Positive: 10594, Negative: 7406
8877 2976
1717 4430


(           clfn3000  clfn3000-279  clfn18000  clfn18000-279  clf18000
 accuracy   0.649333      0.616487   0.726722       0.630824  0.739278
 precision  0.729057      0.688525   0.741757       0.652439  0.748924
 recall     0.582278      0.549020   0.821786       0.699346  0.837927
 f1_score   0.647453      0.610909   0.779723       0.675079  0.790930, None)

In [48]:
# тест 18000 последних  игр
y_pred279 = clf.predict(X279)
metrics(summary,'clf18000-279',y279,y_pred279),conf_mat(y279, y_pred279)

Positive: 153, Negative: 126
113 58
40 68


(           clfn3000  clfn3000-279  ...  clf18000  clf18000-279
 accuracy   0.649333      0.616487  ...  0.739278      0.648746
 precision  0.729057      0.688525  ...  0.748924      0.660819
 recall     0.582278      0.549020  ...  0.837927      0.738562
 f1_score   0.647453      0.610909  ...  0.790930      0.697531
 
 [4 rows x 6 columns], None)

In [49]:
saved_filename = clf.save_model('/content/drive/MyDrive/data/tabnet')

Successfully saved model at /content/drive/MyDrive/data/tabnet.zip


In [50]:
from google.colab import output
output.enable_custom_widget_manager()

In [51]:
X_train,X_valid, y_train, y_valid = train_test_split(X18000, y18000, test_size=0.2)
cbmn = CatBoostClassifier()
cbmn.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)

In [52]:
preds18000 = cbmn.predict(X18000)
# preds18000 = np.rint(preds18000)
accuracy = accuracy_score(y18000, preds18000)
print(accuracy)

0.7269444444444444


In [53]:
# тест 18000 последних  игр
y_pred18000 = cbmn.predict(X18000)
# y_pred18000 = np.rint(y_pred18000)
metrics(summary,'cbmn18000',y18000,y_pred18000),conf_mat(y18000, y_pred18000)

Positive: 10594, Negative: 7406
8678 2999
1916 4407


(           clfn3000  clfn3000-279  clfn18000  ...  clf18000  clf18000-279  cbmn18000
 accuracy   0.649333      0.616487   0.726722  ...  0.739278      0.648746   0.726944
 precision  0.729057      0.688525   0.741757  ...  0.748924      0.660819   0.743170
 recall     0.582278      0.549020   0.821786  ...  0.837927      0.738562   0.819143
 f1_score   0.647453      0.610909   0.779723  ...  0.790930      0.697531   0.779309
 
 [4 rows x 7 columns], None)

In [54]:
# тест 279 последних  игр
y_pred279 = cbmn.predict(X279)

metrics(summary,'cbmn279',y279,y_pred279),conf_mat(y279, y_pred279)

Positive: 153, Negative: 126
107 53
46 73


(           clfn3000  clfn3000-279  clfn18000  ...  clf18000-279  cbmn18000   cbmn279
 accuracy   0.649333      0.616487   0.726722  ...      0.648746   0.726944  0.645161
 precision  0.729057      0.688525   0.741757  ...      0.660819   0.743170  0.668750
 recall     0.582278      0.549020   0.821786  ...      0.738562   0.819143  0.699346
 f1_score   0.647453      0.610909   0.779723  ...      0.697531   0.779309  0.683706
 
 [4 rows x 8 columns], None)

In [55]:
def objective(trial):
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "leaf_estimation_method": trial.suggest_categorical("leaf_estimation_method",['Newton','Gradient']),
        "verbose": False}

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
    '''cross-validation'''
    kf = StratifiedKFold(n_splits=10,random_state=42,shuffle = True)
    CV_score_array =[]
    for train_index, test_index in kf.split(X18000,y18000):
        X_train, X_valid = X18000[train_index], X18000[test_index]
        y_train, y_valid = y18000[train_index], y18000[test_index]
        cbc = CatBoostClassifier(**param)
        cbc.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)

        preds = cbc.predict(X_valid)
        pred_labels = np.rint(preds)
        accuracy = accuracy_score(y_valid, pred_labels)

        CV_score_array.append(accuracy)
    avg = np.mean(CV_score_array)

    return avg

In [56]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout=600)

[I 2022-02-04 00:29:36,933] A new study created in memory with name: no-name-ac24225c-a392-4560-b5af-95aec789b494
[I 2022-02-04 00:33:59,187] Trial 0 finished with value: 0.7079444444444444 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.09926121753192094, 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'leaf_estimation_method': 'Newton'}. Best is trial 0 with value: 0.7079444444444444.
[I 2022-02-04 00:34:32,077] Trial 1 finished with value: 0.7082777777777778 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.05893511663362593, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'leaf_estimation_method': 'Newton', 'subsample': 0.32179063424775967}. Best is trial 1 with value: 0.7082777777777778.
[I 2022-02-04 00:47:50,770] Trial 2 finished with value: 0.7059444444444445 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.05636308839704443, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', '

In [57]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 3
Best trial:
  Value: 0.7082777777777778
  Params: 
    objective: CrossEntropy
    colsample_bylevel: 0.05893511663362593
    depth: 2
    boosting_type: Plain
    bootstrap_type: Bernoulli
    leaf_estimation_method: Newton
    subsample: 0.32179063424775967


In [58]:
X_train,X_valid, y_train, y_valid = train_test_split(X18000, y18000, test_size=0.2)
fcbm = CatBoostClassifier(**trial.params)
fcbm.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)

In [59]:
fcbm.save_model("/content/drive/MyDrive/data/catboost")

catboost = CatBoostClassifier()

catboost.load_model("/content/drive/MyDrive/data/catboost")

In [ ]:
|

In [60]:
y_pred18000 = fcbm.predict(X18000)
metrics(summary,'fcbm18000',y18000,y_pred18000),conf_mat(y18000, y_pred18000)

Positive: 10594, Negative: 7406
8611 3169
1983 4237


(           clfn3000  clfn3000-279  clfn18000  ...  cbmn18000   cbmn279  fcbm18000
 accuracy   0.649333      0.616487   0.726722  ...   0.726944  0.645161   0.713778
 precision  0.729057      0.688525   0.741757  ...   0.743170  0.668750   0.730985
 recall     0.582278      0.549020   0.821786  ...   0.819143  0.699346   0.812819
 f1_score   0.647453      0.610909   0.779723  ...   0.779309  0.683706   0.769733
 
 [4 rows x 9 columns], None)

In [61]:
y_pred279 = fcbm.predict(X279)
metrics(summary,'fcbm279',y279,y_pred279),conf_mat(y279, y_pred279)

Positive: 153, Negative: 126
111 57
42 69


(           clfn3000  clfn3000-279  clfn18000  ...   cbmn279  fcbm18000   fcbm279
 accuracy   0.649333      0.616487   0.726722  ...  0.645161   0.713778  0.645161
 precision  0.729057      0.688525   0.741757  ...  0.668750   0.730985  0.660714
 recall     0.582278      0.549020   0.821786  ...  0.699346   0.812819  0.725490
 f1_score   0.647453      0.610909   0.779723  ...  0.683706   0.769733  0.691589
 
 [4 rows x 10 columns], None)

In [62]:
summary

,clfn3000,clfn3000-279,clfn18000,clfn18000-279,clf18000,clf18000-279,cbmn18000,cbmn279,fcbm18000,fcbm279
accuracy,0.649333,0.616487,0.726722,0.630824,0.739278,0.648746,0.726944,0.645161,0.713778,0.645161
precision,0.729057,0.688525,0.741757,0.652439,0.748924,0.660819,0.743170,0.668750,0.730985,0.660714
recall,0.582278,0.549020,0.821786,0.699346,0.837927,0.738562,0.819143,0.699346,0.812819,0.725490
f1_score,0.647453,0.610909,0.779723,0.675079,0.790930,0.697531,0.779309,0.683706,0.769733,0.691589


In [ ]:
saved_filename = clf.save_model('/content/drive/MyDrive/data/tabnet')
